# enrichissment full #4

In [1]:
import urllib2
from bs4 import BeautifulSoup
import pandas as pd
from decimal import *
import numpy as np
import re
import unicodedata
import csv
from geopy.distance import vincenty

# Récupération des ports

## Source des données : http://www.vesseltracker.com/en/CoveredAreas.html

In [2]:
response = urllib2.urlopen('http://www.vesseltracker.com/en/CoveredAreas.html')
html = response.read()

In [3]:
page = BeautifulSoup(html)

/Users/inesfrebault/app/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [6]:
def recup_href(doc_html):
    with open('../data/referentiels/harbors.csv', 'w') as csvfile:
        fieldnames = ['Harbor', 'Latitude','Longitude']

        writer = csv.DictWriter(csvfile, delimiter=';',fieldnames=fieldnames)
        writer.writeheader()
        
        spamwriter = csv.writer(csvfile, delimiter=';',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        for i in range(13,2003):
            # retrieval of the href of harbors
            stringId = 'theLink_' + str(i)
            for link in doc_html.find_all(id=stringId):
                port = link.get('href')
                port = port.replace(" ","%20")
                string = 'http://www.vesseltracker.com' + port
                harbor = urllib2.urlopen(string)
                html2 = harbor.read()
                page2 = BeautifulSoup(html2)
                stringSplit=string.split('/')
                # retrieval of the coordinates fom the harbor's page
                for a in page2.find_all('div',{'class' : 'info'}):
                    for b in a.find_all('div',{'class' : 'item'}):
                        if (b.get_text().find('Coordinates') > -1):
                            text=a.get_text()
                            text = text.encode('ascii', 'ignore')
                            Arraytext = text.split(":")
                            coord=Arraytext[1]
                            ArrayCoord = coord.split("'")
                            spamwriter.writerow([stringSplit[5],ArrayCoord[0],ArrayCoord[1]])

In [7]:
recup_href(page)

# Conversion

In [8]:
def convert(string):
    stringSplitted = string.split(" ")
    result = float(stringSplitted[1]) + float(stringSplitted[2])/60
    if (stringSplitted[0] == "S" or stringSplitted[0] == "W"):
        result = -result
    return result

In [9]:
dfHarbor = pd.read_csv('../data/referentiels/harbors.csv',sep=';')

In [10]:
dfHarbor['Latitude2'] = dfHarbor.apply(lambda row: convert(row['Latitude']), axis=1)
dfHarbor['Longitude2'] = dfHarbor.apply(lambda row: convert(row['Longitude']), axis=1)

In [11]:
dfHarbor.to_csv("../data/referentiels/harbors.csv",sep=';',index=False, encoding='utf-8')

# Distance

In [12]:
threshold=40

def get_distance(lat, lon,dataset):
    tab=[]
    tab2=[]
    pointAttack=(lat, lon)
    HarborCurrent=dataset[(dataset.Latitude2<lat+threshold) & (dataset.Latitude2>lat-threshold) & (dataset.Longitude2>lon-threshold) & (dataset.Longitude2<lon+threshold)]
    length=len(HarborCurrent)

    for index in range(0,length):
        pointCoast= (HarborCurrent.iloc[index]['Latitude2'], HarborCurrent.iloc[index]['Longitude2'])
        distance =vincenty(pointCoast, pointAttack).kilometers
        tab.append(distance)
        tab2.append(HarborCurrent.iloc[index]['Harbor'])
    
    return min(tab)

#print 'distance : ' +str(get_distance(lat, lon,dfHarbor))

In [13]:
df = pd.read_csv('../data/donneesBrutesPirateriesLatLon.csv',sep=';')
df['distance au port'] = df.apply(lambda row: get_distance(row['latitude'],row['longitude'],dfHarbor), axis=1)

In [14]:
df.to_csv("../data/donneesBrutesPirateriesLatLon.csv",sep=';',index=False, encoding='utf-8')